
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## HyCReeW runup estimation 

inputs required: 
  * Nearshore reconstructed historical storms
  * Nearshore reconstructed simulated storms
  * Historical water levels
  * Synthetic water levels 

in this notebook:
  * HyCReWW runup estimation of historical and synthetic events
  * Extreme value analysis and validation


### Workflow:

<div>
<img src="resources/nb02_04.png" width="400px">
</div>



**HyCReWW** provides wave-driven run-up estimations along coral reef-lined shorelines under a wide range of fringing reef morphologies and offshore forcing characteristics. The metamodel is based on two models: (a) a full factorial design of recent XBeach Non-Hydrostatic simulations under different reef configurations and offshore wave and water level conditions (Pearson et al, 2017); and (b) Radial Basis Functions (RBFs) for approximating the non-linear function of run-up for the set of multivariate parameters:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Runup = RBF($\eta_0$, $H_0$, ${H_0/L_0}$, $\beta_f$,$W_reef$, $\beta_b$, $c_f$ );&nbsp;&nbsp;&nbsp;&nbsp;</center>

Where, the hydrodynamic variables defined are offshore water level ($\eta_0$), significant wave height ($H_0$), and wave steepness (${H_0/L_0}$); the reef morphologic parameters include fore reef slope ($\beta_f$), reef flat width ($W_reef$), beach slope ($\beta_b$), and seabed roughness ($c_f$). ${L_0}$ is the deep water wave length $L_0=gT_p^2/2pi$, and $T_p$ is the peak period. Beach crest elevation ($z_b$) was fixed at a height of 30 m to focus on run-up as a proxy for coastal inundation.

<img src="resources/nb02_04_profile.png">


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize

from teslakit.plotting.extremes import Plot_ReturnPeriodValidation



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')



## HyCREWW - RBFs configuration

runup has been calculated for a total of 15 scenarios (hs, hs_lo) and a set of reef characteristics 


In [3]:
# 15 scenarios of runup model execution

# RBF wave conditions 
rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05]

# load trained RBF coefficients and variables min. and max. limits
var_lims, rbf_coeffs = db.Load_HYCREWW()

# reef characteristics
reef_cs = {
    'rslope': 0.0505,
    'bslope': 0.1667,
    'rwidth': 250,
    'cf': 0.0105,
}

# rbf variables names: level is our teslakit input data
rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'cf']


In [4]:
# run up calculation 

def HyCREWW_RU(df):
    '''
    Calculates runup using HyCREWW RBFs (level, reef variables) 
    and a linear interpolation (hs, hs_lo2) to input dataset
    
    var_lims   - HyCREWW variables min and max limits
    rbf_coeffs - HyCREWW rbf coefficients
    reef_cs    - reef characteristics
    rbf_vns    - rbf variables

    df         - input pandas.dataframe (time,), vars: level, hs, tp, dir, hs_lo2
    '''
    
    
    # 1. Prepare input data
    # -----------------------------------------------------------------
    
    # add reef characteristics to input dataset
    for p in reef_cs.keys(): df[p] = reef_cs[p]
        
    # filter data: all variables inside limits
    lp = []
    for vn in var_lims.keys():
        ps = (df[vn] > var_lims[vn][0]) & (df[vn] < var_lims[vn][1])
        lp.append(ps)
    ix_in = np.where(np.all(lp, axis=0))[0]
        
    # select dataset to interpolate at RBFs
    ds_in = df.iloc[ix_in]
    ds_rbf_in = ds_in[rbf_vns]
    
    
    
    # 2. Calculate RUNUP with input LEVEL for the 15 RBF scenarios
    # -----------------------------------------------------------------   
    
    # parameters
    ix_sc = [0, 1, 2, 3, 4]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]

    # Normalize data
    ds_nm ,_ ,_ = Normalize(ds_rbf_in.values, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolate level for the 15 scenarios
    aux_1 = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        aux_1.append(ro)
    ru_z = np.array(aux_1)

    
    
    # 3. interpolate RUNUP for input WAVES with the 15 RBF scenarios
    # -----------------------------------------------------------------   
     
    # RU linear interpolation (15 sets: hs, hs_lo -> runup)
    #ru_in = np.zeros(ds_in.shape[0]) * np.nan
    #for c, (_, r) in enumerate(ds_in.iterrows()):
    #    ru_in[c] = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (r['hs'], r['hs_lo2']), method='linear')
    
    # RU linear interpolation (15 sets: hs, hs_lo -> runup)  (*faster than loop)
    def axis_ipl_rbfs(inp):
        return griddata((rbf_hs, rbf_hs_lo), inp[:15], (inp[15], inp[16]), method='linear')

    inp = np.concatenate((ru_z, ds_in[['hs', 'hs_lo2']].T))      
    ru_in = np.apply_along_axis(axis_ipl_rbfs, 0, inp)
    
    
    
    # 4. Prepare output 
    # -----------------------------------------------------------------
    
    # add level to run_up
    ru_in = ru_in + ds_in['level']
    
    # return runup 
    ru_out = np.zeros(len(df.index)) * np.nan
    ru_out[ix_in] = ru_in
    
    xds_ru = xr.Dataset({'runup': (('time',), ru_out)}, coords={'time': df.index})

    return xds_ru



## HyCREWW RBF Interpolation: Historical

In [5]:
# Load complete historical data and nearshore waves

# offshore level 
level = db.Load_HIST_OFFSHORE(vns=['level'], decode_times=True)

# nearshore waves
waves = db.Load_HIST_NEARSHORE(vns=['Hs', 'Tp', 'Dir'], decode_times=True)
waves["time"] = waves["time"].dt.round("H")  # fix waves times: round to nearest hour

# use same time for nearshore calculations
level = level.sel(time=waves.time)


# prepare data for hycreww
waves = waves.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
waves['hs_lo2'] = waves['hs']/(1.5613*waves['tp']**2)             # calc. hs_lo2
waves['level'] = level['level']                                   # add level

# calculate runup with hycreww
ru_hist = HyCREWW_RU(waves[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe())

# store historical runup
db.Save_HIST_NEARSHORE(ru_hist)



## HyCREWW RBF Interpolation: Simulation


In [6]:
# offshore level 
level = db.Load_SIM_OFFSHORE_all(vns=['level'], decode_times=False)

# nearshore waves
waves = db.Load_SIM_NEARSHORE_all(vns=['Hs', 'Tp', 'Dir', 'max_storms'], decode_times=False)

# prepare data for hycreww
waves = waves.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
waves['hs_lo2'] = waves['hs']/(1.5613*waves['tp']**2)             # calc. hs_lo2
waves['level'] = level['level']                                   # add level


In [7]:
# TODO: solve optimization problem
import sys; sys.exit()


# iterate  simulations
for n in waves.n_sim:
    waves_n = waves.sel(n_sim=int(n))
    
    # calculate runup with hycreww
    ru_sim_n = HyCREWW_RU(waves_n[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe())

    # store simulation runup
    db.Save_SIM_NEARSHORE(ru_sim_n, int(n))
    
    print('simulation {0} processed.'.format(int(n)))


SystemExit: 


## Methodology Validation: Annual Maxima

In [ ]:
# load all simulations
ru_sims = db.Load_SIM_NEARSHORE_all(vns=['runup'], decode_times=True)


# compare historical and simulations runup annual maxima
hist_A = ru_hist['runup'].groupby('time.year').max(dim='time')
sim_A = ru_sims['runup'].groupby('time.year').max(dim='time')

# Return Period historical vs. simulations
Plot_ReturnPeriodValidation(hist_A, sim_A);
    